# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents

In [2]:
# Run this code:
salaries = pd.read_csv('./Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
salaries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
print(salaries['Salary or Hourly'].value_counts())

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
print(salaries['Department'].value_counts())

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
from statsmodels.stats.weightstats import ztest
# Since we have an N of 8022 and only one sample (Hourly workers), we go for a Z-test
# H0: Hourly rate == 30
# H1: Hourly rate != 30
salaries_hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']

z_statistic, p_value = ztest(salaries_hourly['Hourly Rate'], value=30)

print(f"Z-statistic: {z_statistic}")
print(f"P-value: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("The hourly wage is significantly different from $30/hr.")
else:
    print("The hourly wage is not significantly different from $30/hr.")

Z-statistic: 20.6198057854942
P-value: 1.8228873859286195e-94
The hourly wage is significantly different from $30/hr.


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [8]:
# Subset of the dataset: only Police department
salaries_police = salaries[(salaries['Department'] == 'POLICE') & (salaries['Salary or Hourly'] == 'Salary')]

z_statistic, p_value = ztest(salaries_police['Annual Salary'], value=86000, alternative='larger')
# alternative='larger' because it is a one-tailed test.
# H0: Annual Salary = 86000
# H1: Annual Salary > 86000

print(f"Z-statistic: {z_statistic}")
print(f"P-value: {p_value}")

alpha = 0.05
if p_value < alpha and z_statistic > 0:
    print("H0 rejected: he's right.")
else:
    print("H0 not rejected: he's lying.")


Z-statistic: 3.081997005712994
P-value: 0.0010280845287547853
H0 rejected: he's right.


Using the `crosstab` function, find the department that has the most hourly workers. 

In [9]:
department_most_hourly = pd.crosstab(salaries['Department'], salaries['Salary or Hourly']).sort_values(by='Hourly', ascending=False).head(1)
department_most_hourly

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [10]:
from scipy.stats import  ttest_1samp

hourly_wages_department = salaries[(salaries['Department'] == department_most_hourly.index[0]) &  (salaries['Salary or Hourly'] == 'Hourly')]['Hourly Rate']

t_statistic, p_value = ttest_1samp(hourly_wages_department, 35)
# H0: Hourly rate = 35
# H1: Hourly rate != 35

print(f"T-statistic: {t_statistic}")
print(f"P-value (one-tailed): {p_value / 2}")

alpha = 0.05
if p_value / 2 < alpha and t_statistic < 0:
    print(f"Yes, they are right. Hourly wage in the {department_most_hourly} department is significantly less than $35/hour.")
else:
    print(f"No, it's not true. Hourly wage in the {department_most_hourly} department is not significantly less than $35/hour.")

T-statistic: -9.567447887848152
P-value (one-tailed): 1.6689265282353859e-21
Yes, they are right. Hourly wage in the Salary or Hourly  Hourly  Salary
Department                      
STREETS & SAN       1862     336 department is significantly less than $35/hour.
